In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import json
import requests
import pandas as pd
import pyodbc
import openai
from rich.console import Console
from rich.panel import Panel
from dotenv import load_dotenv

load_dotenv()
console = Console()

# Azure OpenAI configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "your-azure-openai-api-key")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-azure-openai-endpoint.openai.azure.com/")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME", "gpt-4o")  # update as needed

# Azure AI Search configuration
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-search-service.search.windows.net")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY", "your-azure-search-key")
SEARCH_INDEX_NAME = "acc-guidelines-index"


# Bing Search credentials (optional)
BING_SEARCH_API_KEY = os.getenv("BING_SEARCH_API_KEY", "<YOUR-BING-SEARCH-KEY>")
BING_SEARCH_ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"

# Import Azure libraries for search
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizableTextQuery

# Import Azure OpenAI client
from openai import AzureOpenAI

console = Console()

# Azure SQL connection details (for patient data)
server = os.getenv("AZURE_SQL_SERVER_NAME")
database = os.getenv("AZURE_SQL_DATABASE_NAME")
username = os.getenv("AZURE_SQL_USER_NAME")
password = os.getenv("AZURE_SQL_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

# Create an Azure SQL connection string
AZURE_SQL_CONNECTION_STRING = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"


In [3]:
# Initialize the Azure OpenAI client
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
)

In [4]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizableTextQuery

def search_acc_guidelines(query: str) -> str:
    """
    Searches the Azure AI Search index 'acc-guidelines-index' 
    for relevant American College of Cardiology (ACC) guidelines.
    """
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=credential,
    )
    
    results = client.search(
        search_text=query,
        vector_queries=[
            VectorizableTextQuery(
                text=query,
                k_nearest_neighbors=10,   # Adjust as needed
                fields="embedding"   # Adjust based on your index schema
            )
        ],
        query_type="semantic",
        semantic_configuration_name="default",
        search_fields=["chunk"],
        top=10,
        include_total_count=True
    )
    
    retrieved_texts = []
    for result in results:
        content_chunk = result.get("chunk", "")
        retrieved_texts.append(content_chunk)
    
    context_str = "\n".join(retrieved_texts) if retrieved_texts else "No relevant guidelines found."
    
    console.print(
        Panel(
            f"Tool Invoked: ACC Guidelines Search\nQuery: {query}",
            style="bold yellow"
        )
    )
    return context_str


In [5]:
def search_bing(query: str) -> str:
    """
    Searches the public web using the Bing Search API.
    """
    headers = {"Ocp-Apim-Subscription-Key": BING_SEARCH_API_KEY}
    params = {"q": query, "textDecorations": True, "textFormat": "Raw"}
    response = requests.get(BING_SEARCH_ENDPOINT, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if "webPages" in data and "value" in data["webPages"]:
            snippets = [item.get("snippet", "") for item in data["webPages"]["value"]]
            result_text = "\n".join(snippets)
        else:
            result_text = "No Bing results found."
    else:
        result_text = f"Bing search failed with status code {response.status_code}."
    
    console.print(
        Panel(
            f"Tool Invoked: Bing Search\nQuery: {query}",
            style="bold magenta"
        )
    )
    return result_text


In [6]:
import sqlalchemy

def lookup_patient_data(query: str) -> str:
    """
    Queries the 'PatientMedicalData' table in Azure SQL and returns the results as a string.
    'query' should be a valid SQL statement.
    This version uses SQLAlchemy to create an engine, which is fully supported by pandas.read_sql.
    """
    try:
        # Construct the connection URI for SQLAlchemy
        connection_uri = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = sqlalchemy.create_engine(connection_uri)
        
        # Use the engine in pandas.read_sql, which avoids the warning
        df = pd.read_sql(query, engine)
        if df.empty:
            return "No rows found."
        return df.to_string(index=False)
    except Exception as e:
        return f"Database error: {str(e)}"


In [46]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_acc_guidelines",
            "description": "Query the ACC guidelines for official cardiology recommendations. Use keywords related to cardiology conditions, treatments, or guidelines.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Keywords or specific question related to cardiology guidelines."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Perform a public web search for real-time or external information. For example, 'FDA new hyperlipidemia drugs', 'recent hypertension medication approvals'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "General query to retrieve public data."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "lookup_patient_data",
            "description": (
                "Query Azure SQL PatientMedicalData table. Schema: "
                "PatientID: INT (PK, Identity), FirstName: VARCHAR(100), LastName: VARCHAR(100), "
                "DateOfBirth: DATE, Gender: VARCHAR(20), ContactNumber: VARCHAR(100), EmailAddress: VARCHAR(100), "
                "Address: VARCHAR(255), City: VARCHAR(100), PostalCode: VARCHAR(20), Country: VARCHAR(100), "
                "MedicalCondition: VARCHAR(255), Medications: VARCHAR(255), Allergies: VARCHAR(255), BloodType: VARCHAR(10), "
                "LastVisitDate: DATE, SmokingStatus: VARCHAR(50), AlcoholConsumption: VARCHAR(50), ExerciseFrequency: VARCHAR(50), "
                "Occupation: VARCHAR(100), Height_cm: DECIMAL(5,2), Weight_kg: DECIMAL(5,2), BloodPressure: VARCHAR(20), "
                "HeartRate_bpm: INT, Temperature_C: DECIMAL(3,1), Notes: VARCHAR(MAX). Use SQL to retrieve patient data."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Valid SQL query for PatientMedicalData table to get patient information."
                    }
                },
                "required": ["query"]
            }
        }
    }
]

# For convenience, map the tool name to the actual function
tool_implementations = {
    "search_acc_guidelines": search_acc_guidelines,
    "search_bing": search_bing,
    "lookup_patient_data": lookup_patient_data,
}

In [47]:
# ----------------------------
# System Prompt for the Agent
# ----------------------------
SYSTEM_PROMPT = (
    "You are a cardiology-focused AI assistant with access to three tools:\n"
    "1) 'lookup_patient_data' for querying patient records from Azure SQL.\n"
    "2) 'search_acc_guidelines' for official ACC guidelines.\n"
    "3) 'search_bing' for real-time public information.\n\n"
    "You can call these tools in any order, multiple times if needed, to gather all the context.\n"
    "Stop calling tools only when you have enough information to provide a final, cohesive answer.\n"
    "Then output your final answer to the user."
)


In [48]:
# ----------------------------
# Agent Logic (Multi-Step) using new tools syntax
# ----------------------------
def run_multi_step_agent(user_query: str, max_steps: int = 10):
    """
    A multi-step agent with enhanced debugging logs using the new tools syntax.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    for step in range(max_steps):
        console.print(Panel(f"**Step {step+1}**: Starting step {step+1}", title="Step Start", style="bold cyan"))
        
        response = openai_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=8000,
        )
        response_message = response.choices[0].message
        
        console.print(Panel(f"[DEBUG] Step {step+1}: Raw response_message: {response_message}", title="Raw Response", style="dim"))
        
        # IMPORTANT: Append the assistant's message (which includes tool_calls) to the conversation.
        messages.append(response_message)
        
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments_str = tool_call.function.arguments
                
                console.print(Panel(f"[DEBUG] Step {step+1}: Tool called: {function_name}", title="Tool Call Info", style="bold magenta"))
                console.print(Panel(f"[DEBUG] Step {step+1}: arguments_str: {arguments_str}", title="Arguments String", style="magenta"))
                
                if arguments_str.strip() == "":
                    function_args = {}
                else:
                    try:
                        function_args = json.loads(arguments_str)
                        console.print(Panel(f"[DEBUG] Step {step+1}: function_args (JSON parsed): {function_args}", title="Parsed Arguments", style="green"))
                    except json.JSONDecodeError as e:
                        function_args = {}
                        console.print(
                            Panel(
                                "Warning: Could not decode tool call arguments; defaulting to empty dict.",
                                title="JSONDecodeError",
                                style="bold red"
                            )
                        )
                        console.print(Panel(f"[DEBUG] Step {step+1}: JSONDecodeError details: {e}", title="JSONDecodeError Detail", style="bold red"))
                        console.print(Panel(f"[DEBUG] Step {step+1}: function_args (defaulted): {function_args}", title="Defaulted Arguments", style="red"))
                
                # Ensure the 'query' key is present
                if "query" not in function_args or not function_args["query"]:
                    function_args["query"] = user_query
                    console.print(Panel(f"[DEBUG] Step {step+1}: 'query' key defaulted to user_query: {user_query}", title="Defaulted Query Key", style="yellow"))
                
                console.print(
                    Panel(
                        f"**Step {step+1}**: LLM calls tool [bold]{function_name}[/bold]\n\n"
                        f"**Arguments**:\n{json.dumps(function_args, indent=2)}",
                        title="Tool Call",
                        style="bold blue"
                    )
                )
                console.print(Panel(f"[DEBUG] Step {step+1}: Calling tool implementation: {function_name} with args: {function_args}", title="Tool Implementation Call", style="bold magenta"))
                
                tool_fn = tool_implementations.get(function_name)
                if tool_fn is None:
                    tool_output = f"[Error] No implementation for tool '{function_name}'."
                else:
                    tool_output = tool_fn(**function_args)
                console.print(Panel(f"[DEBUG] Step {step+1}: Tool output: {tool_output}", title="Tool Output", style="italic blue"))
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": tool_output,
                })
        else:
            final_answer = response_message.content
            console.print(
                Panel(
                    final_answer,
                    title="Final Answer",
                    style="bold green",
                    border_style="yellow"
                )
            )
            return

    console.print(
        Panel(
            "Max steps reached without a final answer. Stopping.",
            title="Warning",
            style="bold red"
        )
    )
    return

In [53]:
user_question_1 = "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"
run_multi_step_agent(user_question_1)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Raw response_message: ChatCompletionMessage(content=None, refusal=None, role='assistant',       │
│ audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_csPEfaN6YbEmlFZ8NN9OKmNk',   │
│ function=Function(arguments='{"query":"first-line therapy for hypertension in elderly patients"}',              │
│ name='search_acc_guidelines'), type='function')])                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tool Call Info ─────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool called: search_acc_guidelines                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Arguments String ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: arguments_str: {"query":"first-line therapy for hypertension in elderly patients"}              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Parsed Arguments ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: function_args (JSON parsed): {'query': 'first-line therapy for hypertension in elderly          │
│ patients'}                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: ACC Guidelines Search                                                                             │
│ Query: first-line therapy for hypertension in elderly patients                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool output: # Whelton et al.                                                                   │
│                                                                                                                 │
│ # 2017 High Blood Pressure Clinical Practice Guideline                                                          │
│                                                                                                                 │
│ # MAY 15, 2018: e127–248                                                                                        │
│                                                                                                                 │
│ 2. The treatment of patients with hypertension without elevated risk has been systematically understudied       │
│ because lower-risk groups would require prolonged follow-up to have a sufficient number of clinical events to   │
│ provide useful information. Although there is clinical trial evidence that both drug and nondrug therapy will   │
│ interrupt the progressive course of hypertension (S8.1.5-6), there is no trial evidence that this treatment     │
│ decreases CVD morbidity and mortality. The clinical trial evidence is strongest for a target BP of 140/90 mm Hg │
│ in this population. However, observational studies suggest that these individuals often have a high lifetime    │
│ risk and would benefit from BP control earlier in life (S8.1.5-19, S8.1.5-20).                                  │
│                                                                                                                 │
│ # 8.1.6. Choice of Initial Medication                                                                           │
│                                                                                                                 │
│ # Recommendation for Choice of Initial Medication                                                               │
│                                                                                                                 │
│ References that support the recommendation are summarized in Online Data Supplement 27 and Systematic Review    │
│ Report.                                                                                                         │
│                                                                                                                 │
│ |COR|LOE|RECOMMENDATION|                                                                                        │
│ |---|---|---|                                                                                                   │
│ |I|ASR|For initiation of antihypertensive drug therapy, first-line agents include thiazide diuretics, CCBs, and │
│ ACE inhibitors or ARBs. (S8.1.6-1, S8.1.6-2)|                                                                   │
│                                                                                                                 │
│ # Synopsis                                                                                                      │
│                                                                                                                 │
│ The overwhelming majority of persons with BP sufficiently elevated to warrant pharmacological therapy may be    │
│ best treated initially with 2 agents (see Section 8.1.6.1). When initiation of pharmacological therapy with a   │
│ single medication is appropriate, primary consideration should be given to comorbid conditions (e.g., HF, CKD)  │
│ for which specific classes of BP-lowering medication are indicated (see Section 9) (S8.1.6-1). In the largest   │
│ head-to-head comparison of first-step drug therapy for hypertension (S8.1.6-3), the thiazide-type diuretic      │
│ chlorthalidone was superior to the CCB amlodipine and 

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 2: Raw response_message: ChatCompletionMessage(content="According to the ACC guidelines for the    │
│ management of hypertension in elderly patients, the first-line therapy options include:\n\n1. **Thiazide        │
│ Diuretics** - These are often recommended for their effectiveness in lowering blood pressure and providing      │
│ benefits in reducing heart failure risk.\n2. **Calcium Channel Blockers (CCBs)** - These can be utilized as     │
│ first-line agents, especially in patients with specific comorbid conditions that warrant their use.\n3.         │
│ **Angiotensin-Converting Enzyme (ACE) Inhibitors** or **Angiotensin Receptor Blockers (ARBs)** - These are also │
│ considered for initial treatment, particularly in patients with conditions such as heart failure or chronic     │
│ kidney disease that may benefit from them.\n\nThe guidelines emphasize that the choice of therapy should take   │
│ into account the patient's overall health status, comorbidities, and the potential for side effects, especially │
│ in older adults who may be more vulnerable to hypotension and orthostatic changes. \n\nIn practice, many        │
│ patients will require more than one medication to effectively manage their blood pressure, particularly those   │
│ with stage 2 hypertension (BP >20/10 mm Hg above target). In such cases, initiating therapy with two agents is  │
│ recommended.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ According to the ACC guidelines for the management of hypertension in elderly patients, the first-line therapy  │
│ options include:                                                                                                │
│                                                                                                                 │
│ 1. **Thiazide Diuretics** - These are often recommended for their effectiveness in lowering blood pressure and  │
│ providing benefits in reducing heart failure risk.                                                              │
│ 2. **Calcium Channel Blockers (CCBs)** - These can be utilized as first-line agents, especially in patients     │
│ with specific comorbid conditions that warrant their use.                                                       │
│ 3. **Angiotensin-Converting Enzyme (ACE) Inhibitors** or **Angiotensin Receptor Blockers (ARBs)** - These are   │
│ also considered for initial treatment, particularly in patients with conditions such as heart failure or        │
│ chronic kidney disease that may benefit from them.                                                              │
│                                                                                                                 │
│ The guidelines emphasize that the choice of therapy should take into account the patient's overall health       │
│ status, comorbidities, and the potential for side effects, especially in older adults who may be more           │
│ vulnerable to hypotension and orthostatic changes.                                                              │
│                                                                                                                 │
│ In practice, many patients will require more than one medication to effectively manage their blood pressure,    │
│ particularly those with stage 2 hypertension (BP >20/10 mm Hg above target). In such cases, initiating therapy  │
│ with two agents is recommended.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [50]:
user_question_2 = "Are there any recent updates in 2025 on new anticoagulant therapies from the FDA?"
run_multi_step_agent(user_question_2)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Raw response_message: ChatCompletionMessage(content=None, refusal=None, role='assistant',       │
│ audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_P5iQgDKmhzYwP9hKfIlOeLxp',   │
│ function=Function(arguments='{"query":"2025 new anticoagulant therapies FDA updates"}', name='search_bing'),    │
│ type='function')])                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tool Call Info ─────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool called: search_bing                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Arguments String ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: arguments_str: {"query":"2025 new anticoagulant therapies FDA updates"}                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Parsed Arguments ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: function_args (JSON parsed): {'query': '2025 new anticoagulant therapies FDA updates'}          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: 2025 new anticoagulant therapies FDA updates                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool output: No. Drug Name Active Ingredient Approval Date FDA-approved use on approval date* │
│ 5. Romvimza: vimseltinib: 2/14/2025: To treat symptomatic tenosynovial giant cell tumor for which surgical    │
│ ...                                                                                                             │
│ Thrombosis is a potentially fatal condition for which various anticoagulant therapies have been used for    │
│ prevention and treatment. However, bleeding events remain a concern with all anticoagulant drugs. ...       │
│ Development of new anticoagulants targeting coagulation factor XI and prospects for clinical use ... 2025 │
│ Feb 13:S0914-5087(25)00061-9. doi ...                                                                           │
│ Grafapex (treosulfan) Lyophilized Powder for Injection. Company: Medexus Pharmaceuticals Inc. Date of Approval: │
│ January 21, 2025 Treatment for: Stem Cell Transplant Conditioning Grafapex (treosulfan) is an alkylating    │
│ agent indicated in combination with fludarabine as a preparative regimen for allogeneic hematopoietic stem cell │
│ transplantation in adult and pediatric patients one year of age and ...                                         │
│ This article aims to support pharmacists by providing updates on new and anticipated novel drug approvals.  │
│ Summary: Selected drug approvals anticipated in the 12-month period covering the fourth quarter of 2024 through │
│ the third quarter of 2025 are reviewed. The analysis emphasizes drugs expected to have significant clinical   │
│ and financial ...                                                                                               │
│ Researchers have reported that a drug representing a new class of anticoagulants showed an “overwhelming”   │
│ reduction in bleeding events compared with standard-of-care treatment in patients with atrial fibrillation    │
│ (AF). Results of the phase II ‘AZALEA-TIMI 71’ study, published in the New England Journal of Medicine on 22  │
│ January 2025, showed that the monoclonal antibody abelacimab […]                                              │
│ The FDA has several new drug applications on its plate, targeting the treatment of Alzheimer’s,           │
│ Parkinson’s, bipolar depression, migraine, and more. ... in adults with advanced Parkinson’s disease,           │
│ subcutaneous (NDA resubmitted; Update: FDA approved on February 4, 2025) AXS-07 (Symbravo/Axsome          │
│ Therapeutics) for Migraine: A novel, oral ...                                                                   │
│ The rapid evolution of amyloidosis treatment is ushering in an era of precision medicine. Experimental        │
│ therapies like CRISPR-Cas 9 gene editing and monoclonal antibodies promise targeted interventions with        │
│ transformative potential. However, challenges remain, including high drug costs and disparities in diagnosis    │
│ and treatment access.                                                                                         │
│ The newly developed factor XIa and XIIa inhibitors have the potential to address these unmet clinical needs and │
│ are undergoing clinical evaluation for several indications. SIGNIFICANCE STATEMENT: Anticoagulant therapy   │
│ is the cornerstone of treatment and prevention of thrombosis, which remains a leading cause of morbidity and  │
│ mortality worldwide.                                                                                            │
│ This has led to the concept of targeting FXI/FXIa as an anticoagulant strategy. Proof of concept is supported │
│ by 20 years of animal model data. Strategies include parenteral monoclonal antibodies to decrease FXI and FXIa  │
│ levels, antisense oligonucleotides, as well as oral sm

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 2: Raw response_message: ChatCompletionMessage(content='In 2025, there have been notable updates   │
│ regarding new anticoagulant therapies, particularly the monoclonal antibody abelacimab, which demonstrated a    │
│ significant reduction in bleeding events compared to standard treatment in patients with atrial fibrillation    │
│ (AF). The results of the phase II study, known as ‘AZALEA-TIMI 71’, were published on January 22, 2025, and     │
│ highlighted its potential advantages over conventional therapies.\n\nAdditionally, there is ongoing development │
│ of novel anticoagulants targeting coagulation factor XI, which may address unmet clinical needs. These new      │
│ therapies include factor XIa and XIIa inhibitors, with various delivery methods, such as parenteral monoclonal  │
│ antibodies, antisense oligonucleotides, and oral small molecule inhibitors, being evaluated in clinical         │
│ studies.\n\nOverall, the landscape for anticoagulant treatments is evolving rapidly, with promising newcomers   │
│ showing potential for enhanced safety and efficacy in managing thrombosis-related conditions.', refusal=None,   │
│ role='assistant', audio=None, function_call=None, tool_calls=None)                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ In 2025, there have been notable updates regarding new anticoagulant therapies, particularly the monoclonal     │
│ antibody abelacimab, which demonstrated a significant reduction in bleeding events compared to standard         │
│ treatment in patients with atrial fibrillation (AF). The results of the phase II study, known as ‘AZALEA-TIMI   │
│ 71’, were published on January 22, 2025, and highlighted its potential advantages over conventional therapies.  │
│                                                                                                                 │
│ Additionally, there is ongoing development of novel anticoagulants targeting coagulation factor XI, which may   │
│ address unmet clinical needs. These new therapies include factor XIa and XIIa inhibitors, with various delivery │
│ methods, such as parenteral monoclonal antibodies, antisense oligonucleotides, and oral small molecule          │
│ inhibitors, being evaluated in clinical studies.                                                                │
│                                                                                                                 │
│ Overall, the landscape for anticoagulant treatments is evolving rapidly, with promising newcomers showing       │
│ potential for enhanced safety and efficacy in managing thrombosis-related conditions.                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [54]:
user_question_3 = "How many patients have Hypertension and are prescribed Lisinopril?"
# The agent should generate a valid SQL query, for example:
## Note, the answer will depend on the actual data in the database (it should return 1071!)
run_multi_step_agent(user_question_3)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Raw response_message: ChatCompletionMessage(content=None, refusal=None, role='assistant',       │
│ audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0ORKc1HyJ8v5OHtod3v3lYHx',   │
│ function=Function(arguments='{"query":"SELECT COUNT(*) as PatientCount FROM PatientMedicalData WHERE            │
│ MedicalCondition LIKE \'%Hypertension%\' AND Medications LIKE \'%Lisinopril%\'"}', name='lookup_patient_data'), │
│ type='function')])                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tool Call Info ─────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool called: lookup_patient_data                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Arguments String ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: arguments_str: {"query":"SELECT COUNT(*) as PatientCount FROM PatientMedicalData WHERE          │
│ MedicalCondition LIKE '%Hypertension%' AND Medications LIKE '%Lisinopril%'"}                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Parsed Arguments ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: function_args (JSON parsed): {'query': "SELECT COUNT(*) as PatientCount FROM PatientMedicalData │
│ WHERE MedicalCondition LIKE '%Hypertension%' AND Medications LIKE '%Lisinopril%'"}                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────── Tool Implementation Call ────────────────────────────────────────────╮
│ [DEBUG] Step 1: Calling tool implementation: lookup_patient_data with args: {'query': "SELECT COUNT(*) as       │
│ PatientCount FROM PatientMedicalData WHERE MedicalCondition LIKE '%Hypertension%' AND Medications LIKE          │
│ '%Lisinopril%'"}                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool output:  PatientCount                                                                      │
│          1071                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 2: Raw response_message: ChatCompletionMessage(content='There are 1,071 patients who have          │
│ Hypertension and are prescribed Lisinopril.', refusal=None, role='assistant', audio=None, function_call=None,   │
│ tool_calls=None)                                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ There are 1,071 patients who have Hypertension and are prescribed Lisinopril.                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [52]:
run_multi_step_agent("I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there are any new medication updates from the FDA as of Feb 2025? Then give me a summary.")

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Raw response_message: ChatCompletionMessage(content=None, refusal=None, role='assistant',       │
│ audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PoMgQiew09X2gAeYSWpObYII',   │
│ function=Function(arguments='{"query": "SELECT * FROM PatientMedicalData WHERE FirstName = \'Gloria\' AND       │
│ LastName = \'Paul\'"}', name='lookup_patient_data'), type='function'),                                          │
│ ChatCompletionMessageToolCall(id='call_CaajWhmNyJ8is5hxgf9RsYtb', function=Function(arguments='{"query":        │
│ "hyperlipidemia guidelines"}', name='search_acc_guidelines'), type='function'),                                 │
│ ChatCompletionMessageToolCall(id='call_2ZOnruZOBA9FyrQ3n4OR8c64', function=Function(arguments='{"query": "FDA   │
│ medication updates hyperlipidemia February 2025"}', name='search_bing'), type='function')])                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tool Call Info ─────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool called: lookup_patient_data                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Arguments String ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: arguments_str: {"query": "SELECT * FROM PatientMedicalData WHERE FirstName = 'Gloria' AND       │
│ LastName = 'Paul'"}                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Parsed Arguments ────────────────────────────────────────────────╮
│ [DEBUG] Step 1: function_args (JSON parsed): {'query': "SELECT * FROM PatientMedicalData WHERE FirstName =      │
│ 'Gloria' AND LastName = 'Paul'"}                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tool Invoked: Bing Search                                                                                       │
│ Query: FDA medication updates hyperlipidemia February 2025                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ [DEBUG] Step 1: Tool output: The Prescription Drug User Fee Act (PDUFA) date refers to the deadline set by    │
│ the US Food and Drug Administration (FDA) for reviewing a New Drug Application (NDA) or Biologics License │
│ Application (BLA) and making a final decision on marketing approval. The typical period for review is 10 months │
│ after the drug application has been accepted by the Agency.                                                   │
│ CINCINNATI- February 10, 2025 --LIB Therapeutics Inc. (LIB), a privately-held, late-stage biopharmaceutical │
│ company today announced the U.S. Food and Drug Administration (FDA) has accepted for review the Biologics   │
│ License Application (BLA) of Lerodalcibep to reduce low-density lipoprotein cholesterol (LDL-C) for the         │
│ treatment of patients with atherosclerotic cardiovascular disease (ASCVD), or ...                               │
│ The FDA has accepted for review the biologics license application (BLA) of lerodalcibep (LIB Therapeutics) to │
│ reduce low-density lipoprotein cholesterol (LDL-C) in the treatment of patients with atherosclerotic            │
│ cardiovascular disease (ASCVD), or very high or high risk of ASCVD, and primary hyperlipidemia, including     │
│ heterozygous patients and those 10 years or older with homozygous familial ...                                  │
│ Topic Guidance Status Date; Clinical - Medical: Obesity and Overweight: Developing Drugs and Biological       │
│ Products for Weight Reduction: Draft: 1/7/2025: Clinical/Medical                                              │
│ The US Food and Drug Administration (FDA) granted an updated indication for bempedoic acid (Nexletol,       │
│ Esperion Therapeutics Inc) and bempedoic acid plus ezetimibe (Nexlizet, Esperion Therapeutics Inc) for the      │
│ treatment of primary hyperlipidemia in eligible patients with genetic hyperlipidemia or atherosclerotic     │
│ cardiovascular disease, according to an Esperion press release.                                                 │
│ The FDA approved an updated indication for Esperion’s bempedoic acid tablet (Nexletol) and bempedoic acid and │
│ ezetimibe tablet (Nexlizet) to include treatment of primary hyperlipidemia as a qualifier for existing        │
│ approved populations, the pharmaceutical company announced in a press release. 1 In addition to the updated     │
│ indication, the federal agency also removed the maximally tolerated ...                                         │
│ In June 2023, Leqembi (lecanemab) became the first fully FDA-approved medication of its kind of Alzheimer’s │
│ disease. It’s thought to work by targeting beta-amyloid plaques to slow the progression of the disease. Fast    │
│ forward to January 2025, and the FDA approved a once-monthly maintenance dosage of Leqembi. This gave       │
│ people receiving ...                                                                                            │
│ Bempedoic acid was first approved in 2020 to lower LDL cholesterol in adults with heterozygous familial         │
│ hypercholesterolemia or established atherosclerotic cardiovascular disease. 2 Then in December 2023, the FDA  │
│ approved a label expansion for bempedoic acid tablet and bempedoic acid and ezetimibe tablet to include         │
│ treatment of primary hyperlipidemia as a qualifier for existing approved ...                                  │
│ On 21 February 2025, the FDA announced that the US semaglutide injection product shortage has resolved,   │
│ such that semaglutide can now be removed from the drug shortages list.The FDA has confirmed ...             │
│ To help stakeholders better understand the guidance, the FDA will host a webinar on March 4, 2025. • Read   │
│ the final guidance here. Conclusion. The breadth of th

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ [DEBUG] Step 2: Raw response_message: ChatCompletionMessage(content='### Summary for Patient Gloria             │
│ Paul\n\n#### Patient Medical Details:\n- **Name:** Gloria Paul\n- **Date of Birth:** July 15, 1944 (79 years    │
│ old)\n- **Gender:** Other\n- **Location:** Lewisberg, MN, India\n- **Medical Condition:** Hyperlipidemia\n-     │
│ **Medications:** Atorvastatin\n- **Allergies:** None\n- **Blood Type:** O+\n- **Last Visit Date:** March 26,    │
│ 2024\n- **Lifestyle Factors:**\n  - **Smoking Status:** Never smoker\n  - **Alcohol Consumption:** Light        │
│ drinker (1-2 times a week)\n  - **Exercise Frequency:** Regular exercise\n\n#### ACC Guidelines for             │
│ Hyperlipidemia:\n1. **Class I Recommendations:**\n   - Statin treatment is recommended for all patients with    │
│ atherosclerosis (including carotid or vertebral atherosclerosis), aiming to reduce LDL cholesterol below 100    │
│ mg/dL.\n   \n2. **Class IIa Recommendations:**\n   - Statins may also be considered for ischemic stroke         │
│ patients to achieve LDL levels around or below 70 mg/dL.\n   - For patients not achieving desired LDL levels    │
│ with statins, additional medications such as bile acid sequestrants or niacin may be beneficial.\n   -          │
│ Alternatives to statins, like bile acid sequestrants, are reasonable for those intolerant to statins.\n\n3.     │
│ **Lifestyle Modifications:** Address lifestyle factors, including obesity or metabolic syndrome, alongside any  │
│ underlying secondary conditions.\n\n#### FDA Updates (as of February 2025):\n- **Lerodalcibep**: A new          │
│ biologics license application has been submitted to the FDA targeting low-density lipoprotein cholesterol       │
│ (LDL-C) for the treatment of patients with atherosclerotic cardiovascular disease or those with primary         │
│ hyperlipidemia.\n  \n- **Bempedoic Acid Update**: The FDA has updated the indication for bempedoic acid         │
│ (Nexletol) to treat primary hyperlipidemia among eligible patients with genetic hyperlipidemia or               │
│ atherosclerotic cardiovascular disease.\n\n### Conclusion:\nFor Mrs. Gloria Paul, FDA updates indicate the      │
│ potential introduction of new medications for managing hyperlipidemia that could complement or enhance her      │
│ current treatment with atorvastatin. Based on the ACC guidelines, continuing atorvastatin is appropriate, with  │
│ a focus on achieving optimal LDL cholesterol levels. Regular check-ups and monitoring of her lipid profile      │
│ would be essential for optimizing her treatment plan.', refusal=None, role='assistant', audio=None,             │
│ function_call=None, tool_calls=None)                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ ### Summary for Patient Gloria Paul                                                                             │
│                                                                                                                 │
│ #### Patient Medical Details:                                                                                   │
│ - **Name:** Gloria Paul                                                                                         │
│ - **Date of Birth:** July 15, 1944 (79 years old)                                                               │
│ - **Gender:** Other                                                                                             │
│ - **Location:** Lewisberg, MN, India                                                                            │
│ - **Medical Condition:** Hyperlipidemia                                                                         │
│ - **Medications:** Atorvastatin                                                                                 │
│ - **Allergies:** None                                                                                           │
│ - **Blood Type:** O+                                                                                            │
│ - **Last Visit Date:** March 26, 2024                                                                           │
│ - **Lifestyle Factors:**                                                                                        │
│   - **Smoking Status:** Never smoker                                                                            │
│   - **Alcohol Consumption:** Light drinker (1-2 times a week)                                                   │
│   - **Exercise Frequency:** Regular exercise                                                                    │
│                                                                                                                 │
│ #### ACC Guidelines for Hyperlipidemia:                                                                         │
│ 1. **Class I Recommendations:**                                                                                 │
│    - Statin treatment is recommended for all patients with atherosclerosis (including carotid or vertebral      │
│ atherosclerosis), aiming to reduce LDL cholesterol below 100 mg/dL.                                             │
│                                                                                                                 │
│ 2. **Class IIa Recommendations:**                                                                               │
│    - Statins may also be considered for ischemic stroke patients to achieve LDL levels around or below 70       │
│ mg/dL.                                                                                                          │
│    - For patients not achieving desired LDL levels with statins, additional medications such as bile acid       │
│ sequestrants or niacin may be beneficial.                                                                       │
│    - Alternatives to statins, like bile acid sequestrants, are reasonable for those intolerant to statins.      │
│                                                                                                                 │
│ 3. **Lifestyle Modifications:** Address lifestyle factors, including obesity or metabolic syndrome, alongside   │
│ any underlying secondary conditions.                                                                            │
│                                                                                                                 │
│ #### FDA Updates (as of February 2025):                                                                         │
│ - **Lerodalcibep**: A new biologics license applicatio